# Stepwise Regression
# Lasso Regression
# Elastic Net

*Using the crime data set from http://www.statsci.org/data/general/uscrime.txt data dictionary at http://www.statsci.org/data/general/uscrime.html, I will build regression	models using:*
1. *Stepwise regression*
2. *Lasso*
3. *Elastic net*



### Load libraries and set seed



In [1]:
options(jupyter.plot_mimetypes = c("text/plain", "image/png" ))
library(glmnet)
library(caret)
set.seed(37)

Loading required package: Matrix
Loading required package: foreach
Loaded glmnet 2.0-13

Loading required package: lattice
Loading required package: ggplot2




### Load data



In [2]:
data <- read.table('uscrime.txt', header = TRUE)



### Check head of data



In [3]:
head(data)

M,So,Ed,Po1,Po2,LF,M.F,Pop,NW,U1,U2,Wealth,Ineq,Prob,Time,Crime
15.1,1,9.1,5.8,5.6,0.510,95.0,33,30.1,0.108,4.1,3940,26.1,0.084602,26.2011,791
14.3,0,11.3,10.3,9.5,0.583,101.2,13,10.2,0.096,3.6,5570,19.4,0.029599,25.2999,1635
14.2,1,8.9,4.5,4.4,0.533,96.9,18,21.9,0.094,3.3,3180,25.0,0.083401,24.3006,578
13.6,0,12.1,14.9,14.1,0.577,99.4,157,8.0,0.102,3.9,6730,16.7,0.015801,29.9012,1969
14.1,0,12.1,10.9,10.1,0.591,98.5,18,3.0,0.091,2.0,5780,17.4,0.041399,21.2998,1234
12.1,0,11.0,11.8,11.5,0.547,96.4,25,4.4,0.084,2.9,6890,12.6,0.034201,20.9995,682




### Check str of data



In [4]:
str(data)

'data.frame':	47 obs. of  16 variables:
 $ M     : num  15.1 14.3 14.2 13.6 14.1 12.1 12.7 13.1 15.7 14 ...
 $ So    : int  1 0 1 0 0 0 1 1 1 0 ...
 $ Ed    : num  9.1 11.3 8.9 12.1 12.1 11 11.1 10.9 9 11.8 ...
 $ Po1   : num  5.8 10.3 4.5 14.9 10.9 11.8 8.2 11.5 6.5 7.1 ...
 $ Po2   : num  5.6 9.5 4.4 14.1 10.1 11.5 7.9 10.9 6.2 6.8 ...
 $ LF    : num  0.51 0.583 0.533 0.577 0.591 0.547 0.519 0.542 0.553 0.632 ...
 $ M.F   : num  95 101.2 96.9 99.4 98.5 ...
 $ Pop   : int  33 13 18 157 18 25 4 50 39 7 ...
 $ NW    : num  30.1 10.2 21.9 8 3 4.4 13.9 17.9 28.6 1.5 ...
 $ U1    : num  0.108 0.096 0.094 0.102 0.091 0.084 0.097 0.079 0.081 0.1 ...
 $ U2    : num  4.1 3.6 3.3 3.9 2 2.9 3.8 3.5 2.8 2.4 ...
 $ Wealth: int  3940 5570 3180 6730 5780 6890 6200 4720 4210 5260 ...
 $ Ineq  : num  26.1 19.4 25 16.7 17.4 12.6 16.8 20.6 23.9 17.4 ...
 $ Prob  : num  0.0846 0.0296 0.0834 0.0158 0.0414 ...
 $ Time  : num  26.2 25.3 24.3 29.9 21.3 ...
 $ Crime : int  791 1635 578 1969 1234 682 963 1555 



## 1. Stepwise Regression

First let's perform the stepwise regression. Stepwise regression is a ***greedy algoithm*** ie it is a brute force algorithm that trys all possible regression models (combination of predictive features), calculates the model prediction error, and picks the model (combination of predictive features) that minimizes the prediction error most. From Dr. Sokol's suggestions we'll use stepwise regression as a kind of quick pass at feature selection. More advanced feature selection models that utilize optimization to pick the best model we'll be demonstrated in the sections detailing the use of Lasso regression and Elastic Net.





### Perform the stepwise regression using `step`

Our first regression model in our stepwise regression model selection process is the null model that does not include any predictive features and is only a function of the y-intercept. Naturally, we should expect this model to perform horribly as it is basically a straight line at y=1. The final model we will try is the model containing all predictive features. The `step` function in R will add a single predictive feature at every "step", exhausting all predictive feature options until the AIC increases (indicative of a poorer model). Luckily for us, the `step` function in R does all of the model performance comparing under the hood. The final model spit out by the `step` model is the optimal model. This final model will be tested using one final round of 10-fold cross-validation. At this point I will report the final cross-validated model performance. See references below for more on the `step` function:

reference: http://www.stat.columbia.edu/~martin/W2024/R10.pdf

http://rstudio-pubs-static.s3.amazonaws.com/2899_a9129debf6bd47d2a0501de9c0dc583d.html

https://www.rdocumentation.org/packages/stats/versions/3.4.3/topics/step

In [5]:
null_features <- lm(Crime~1,data=data)
all_features <- lm(Crime~.,data=data)
stepwise_regression_aic <- step(null_features,scope=list(lower=null_features,upper=all_features),
                            direction="both", trace=1, k=2)

Start:  AIC=561.02
Crime ~ 1

         Df Sum of Sq     RSS    AIC
+ Po1     1   3253302 3627626 532.94
+ Po2     1   3058626 3822302 535.39
+ Wealth  1   1340152 5540775 552.84
+ Prob    1   1257075 5623853 553.54
+ Pop     1    783660 6097267 557.34
+ Ed      1    717146 6163781 557.85
+ M.F     1    314867 6566061 560.82
<none>                6880928 561.02
+ LF      1    245446 6635482 561.32
+ Ineq    1    220530 6660397 561.49
+ U2      1    216354 6664573 561.52
+ Time    1    154545 6726383 561.96
+ So      1     56527 6824400 562.64
+ M       1     55084 6825844 562.65
+ U1      1     17533 6863395 562.90
+ NW      1      7312 6873615 562.97

Step:  AIC=532.94
Crime ~ Po1

         Df Sum of Sq     RSS    AIC
+ Ineq    1    739819 2887807 524.22
+ M       1    616741 3010885 526.18
+ M.F     1    250522 3377104 531.57
+ NW      1    232434 3395192 531.82
+ So      1    219098 3408528 532.01
+ Wealth  1    180872 3446754 532.53
<none>                3627626 532.94
+ Po2     1  

In [6]:
stepwise_regression_aic$anova

Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,46,6880928,561.0235
+ Po1,-1,3253301.8,45,3627626,532.9352
+ Ineq,-1,739818.6,44,2887807,524.2154
+ Ed,-1,587049.8,43,2300757,515.5343
+ M,-1,239404.6,42,2061353,512.3701
+ Prob,-1,258062.5,41,1803290,508.0839
+ U2,-1,192233.4,40,1611057,504.7859




As we can see the AIC is going down for each variable, indicating better model performance with each `step`.



In [7]:
summary(stepwise_regression_aic)


Call:
lm(formula = Crime ~ Po1 + Ineq + Ed + M + Prob + U2, data = data)

Residuals:
    Min      1Q  Median      3Q     Max 
-470.68  -78.41  -19.68  133.12  556.23 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -5040.50     899.84  -5.602 1.72e-06 ***
Po1           115.02      13.75   8.363 2.56e-10 ***
Ineq           67.65      13.94   4.855 1.88e-05 ***
Ed            196.47      44.75   4.390 8.07e-05 ***
M             105.02      33.30   3.154  0.00305 ** 
Prob        -3801.84    1528.10  -2.488  0.01711 *  
U2             89.37      40.91   2.185  0.03483 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 200.7 on 40 degrees of freedom
Multiple R-squared:  0.7659,	Adjusted R-squared:  0.7307 
F-statistic: 21.81 on 6 and 40 DF,  p-value: 3.418e-11


In [8]:
# optimal model prediction accuracy using AIC model selection criteria

summary(stepwise_regression_aic)$adj.r.squared

[1] 0.7307463

In [9]:
# optimal model prediction error using AIC model selection criteria

1 - summary(stepwise_regression_aic)$adj.r.squared

[1] 0.2692537



### Using BIC to select optimal model

We can just as easily use Bayesian Information Criteria (BIC) to select ou optimal model. We just change the value of k from k = 2 to k = log(n). We have 47 data points so n = 47 in this case.



In [10]:
stepwise_regression_bic <- step(null_features,scope=list(lower=null_features,upper=all_features),
                            direction="both", trace=1, k=log(47))

Start:  AIC=562.87
Crime ~ 1

         Df Sum of Sq     RSS    AIC
+ Po1     1   3253302 3627626 536.64
+ Po2     1   3058626 3822302 539.09
+ Wealth  1   1340152 5540775 556.54
+ Prob    1   1257075 5623853 557.24
+ Pop     1    783660 6097267 561.04
+ Ed      1    717146 6163781 561.55
<none>                6880928 562.87
+ M.F     1    314867 6566061 564.52
+ LF      1    245446 6635482 565.02
+ Ineq    1    220530 6660397 565.19
+ U2      1    216354 6664573 565.22
+ Time    1    154545 6726383 565.66
+ So      1     56527 6824400 566.34
+ M       1     55084 6825844 566.35
+ U1      1     17533 6863395 566.60
+ NW      1      7312 6873615 566.67

Step:  AIC=536.64
Crime ~ Po1

         Df Sum of Sq     RSS    AIC
+ Ineq    1    739819 2887807 529.77
+ M       1    616741 3010885 531.73
<none>                3627626 536.64
+ M.F     1    250522 3377104 537.12
+ NW      1    232434 3395192 537.37
+ So      1    219098 3408528 537.56
+ Wealth  1    180872 3446754 538.08
+ Po2     1  

In [11]:
stepwise_regression_bic$anova

Step,Df,Deviance,Resid. Df,Resid. Dev,AIC
,NA,NA,46,6880928,562.8736
+ Po1,-1,3253301.8,45,3627626,536.6355
+ Ineq,-1,739818.6,44,2887807,529.7659
+ Ed,-1,587049.8,43,2300757,522.9349
+ M,-1,239404.6,42,2061353,521.6208
+ Prob,-1,258062.5,41,1803290,519.1848
+ U2,-1,192233.4,40,1611057,517.7369


In [12]:
# optimal model prediction accuracy using BIC model selection criteria

summary(stepwise_regression_bic)$adj.r.squared

[1] 0.7307463

In [13]:
# optimal model prediction error using BIC model selection criteria

1 - summary(stepwise_regression_bic)$adj.r.squared

[1] 0.2692537

In [14]:
summary(stepwise_regression_bic)$adj.r.squared == summary(stepwise_regression_aic)$adj.r.squared

[1] TRUE



We get the exact same performance using the model selected by BIC as AIC criteria. If using less factors is preferred, usually  BIC is the selection criteria to go with.





### Cross-validation on final model chosen by stepwise regression

Now that we have a winner-winner chicken dinner model, let's use leave-one-out cross-validation to report a more "realistic" model performance. "LOOCV" is the `trainControl` parameter we use for leave-one-out cross-validation.



In [48]:
# set cross_validation parameters
train_control<- trainControl(method="LOOCV")

# train model using caret
reduced_regression_stepwise_features_cv <- train(formula(stepwise_regression_bic),data,method = "lm",trControl=train_control)

In [49]:
reduced_regression_stepwise_features_cv

Linear Regression 

47 samples
 6 predictor

No pre-processing
Resampling: Leave-One-Out Cross-Validation 
Summary of sample sizes: 46, 46, 46, 46, 46, 46, ... 
Resampling results:

  RMSE      Rsquared   MAE     
  221.0758  0.6706867  165.5131

Tuning parameter 'intercept' was held constant at a value of TRUE



There you have it! We have successfully used step regression to find our optimal predictive features:

* Po1
* Ineq
* Ed
* M
* Prob
* U2



## 2. Lasso Regression

Recall from the lessons that Lasso regression is a variable selection model that solves a constrained optimization problem under the hood to pick the best subset of available predictive features:

$$
Min \: \: \:  \sum_{i=1}^{n} \left ( y_{i} -\left ( a_{0} + a_{1}x_{1i} + a_{2}x_{2i} + \cdots +a_{j}x_{ij}\right )\right)^{2}
$$
$$
S.t. \: \: \:  \sum_{i=1}^{j} \left | a_{i} \right | \, \leq \tau
$$

Basically our coefficients are constrained by the parameter tau. We parameterize tau and try different values to acheive an optimal model. Luckily for us, Lasso regression can be implemented using the `glmnet` function in R. Varying the value of tau and observing the change in model performance is done under the hood for us. To perform Lasso regression using the `glmnet` function, we hold our parameter `alpha` equal to 1. Before we can use the `glmnet` function, we must scale our data. To accomplish this, I will use z-scaling. 





### Scale the data

To insure that model coefficients are on the same scale, let's first z-scale each ***numerical*** feature in our dataset. Z-scaling insures that our mean is 0 and our standard deviation is 1 for each feature in the dataset. The formula for z-scaling can be found below:

$$
z = \frac{\left (x - \mu_{p}   \right )}{\sigma_{p}}
$$

where p is the feature being scaled. Once we have our scaled ***numerical*** features, let's add back in our response and our categorical variables.



In [40]:
scaled_data <- as.data.frame(apply(data[,c(-2,-16)], 2, scale))
scaled_data <- cbind(scaled_data, data$So, data$Crime)
colnames(scaled_data)[15] <- "So"
colnames(scaled_data)[16] <- "Crime"
scaled_data <- scaled_data[,c(1,15,2,3,4,5,6,7,8,9,10,11,12,13,14,16)]
head(scaled_data)

M,So,Ed,Po1,Po2,LF,M.F,Pop,NW,U1,U2,Wealth,Ineq,Prob,Time,Crime
0.9886930,1,-1.3085099,-0.9085105,-0.8666988,-1.2667456,-1.12060499,-0.09500679,1.943738564,0.69510600,0.8313680,-1.3616094,1.6793638,1.6497631,-0.05599367,791
0.3521372,0,0.6580587,0.6056737,0.5280852,0.5396568,0.98341752,-0.62033844,0.008483424,0.02950365,0.2393332,0.3276683,0.0000000,-0.7693365,-0.18315796,1635
0.2725678,1,-1.4872888,-1.3459415,-1.2958632,-0.6976051,-0.47582390,-0.48900552,1.146296747,-0.08143007,-0.1158877,-2.1492481,1.4036474,1.5969416,-0.32416470,578
-0.2048491,0,1.3731746,2.1535064,2.1732150,0.3911854,0.37257228,3.16204944,-0.205464381,0.36230482,0.5945541,1.5298536,-0.6767585,-1.3761895,0.46611085,1969
0.1929983,0,1.3731746,0.8075649,0.7426673,0.7376187,0.06714965,-0.48900552,-0.691709391,-0.24783066,-1.6551781,0.5453053,-0.5013026,-0.2503580,-0.74759413,1234
-1.3983912,0,0.3898903,1.1104017,1.2433590,-0.3511718,-0.64550313,-0.30513945,-0.555560788,-0.63609870,-0.5895155,1.6956723,-1.7044289,-0.5669349,-0.78996812,682




### Cross-validated Lasso Regression using `glmnet`

In order to use `glmnet` we must first transform our underlying data object to a matrix rather than a dataframe. I will use 5-fold cross-validation in an effort to eliminate bias.



In [41]:
feature_matrix <- as.matrix(scaled_data[-16])
response_matrix <- as.matrix(scaled_data$Crime)

In [50]:
Lasso_regression <- cv.glmnet(x=feature_matrix, y=response_matrix, alpha=1, nfolds=5, type.measure='mse',family='gaussian')



### Coefficients returned by Lasso regression

We can use the `coef` function to examine the coefficients returned by our Lasso regression model.



In [51]:
coef(Lasso_regression, s=Lasso_regression$lambda.min)

16 x 1 sparse Matrix of class "dgCMatrix"
                    1
(Intercept) 894.04865
M           103.66132
So           32.41958
Ed          173.09241
Po1         296.62268
Po2           .      
LF            .      
M.F          52.65592
Pop         -17.56833
NW           13.84601
U1          -69.23520
U2          113.82743
Wealth       50.96794
Ineq        246.80851
Prob        -88.83419
Time          .      



### Linear regression with significant variables returned by Lasso

Now that our Lasso regression has found the optimal subset of predictive features, let's build a linear regression model and observe the model performance measured as R*2.



In [70]:
reduced_regression_Lasso_features <- train(Crime ~ M+So+Ed+Po1+M.F+Pop+NW+U1+U2+Wealth+Ineq+Prob,data,method = "lm",trControl=train_control)
reduced_regression_Lasso_features

Linear Regression 

47 samples
12 predictors

No pre-processing
Resampling: Leave-One-Out Cross-Validation 
Summary of sample sizes: 46, 46, 46, 46, 46, 46, ... 
Resampling results:

  RMSE      Rsquared   MAE     
  244.8839  0.6048887  194.1645

Tuning parameter 'intercept' was held constant at a value of TRUE

In [ ]:
summary(reduced_regression_Lasso_features)



Our linear regression model chosen by the Lasso regression predictive feature selection process performs slightly worse then the model chosen by the stepwise regression predictive feature selection approach. However, notice the significant features. If we remove all non-significant features from our model we are left with the exact same optimal model outputted by our stepwise regression predictive feature selection approach! Using a greedy approach like stepwise regression provides a nice sanity check for any model's outputted by constrained optimization driven approaches like Lasso regression (for smallish datasets where a greedy approach is not too terribly computationally expensive). This specific dataset may be an exception to the general rule of thumb, but as Dr. Sokol pointed out in the lectures, constrained optimization driven methods like Lasso regression often perform better then greedy approaches. This concludes the Lasso regression section.





## 3. Elastic Net

Now that we have looked at stepwise regression and Lasso regression, let's take a gander at the elastic net capability contained within the `glmnet` function. The only difference between Lasso and elastic net within the context of the `glmnet` is our choice of the parameter `alpha`. For elastic net we vary `alpha` to try and solve the constrained optimization problem:

$$
Min \: \: \:  \sum_{i=1}^{n} \left ( y_{i} -\left ( a_{0} + a_{1}x_{1i} + a_{2}x_{2i} + \cdots +a_{j}x_{ij}\right )\right)^{2}
$$
$$
S.t.\: \: \: \lambda \sum_{i=1}^{j} \left | a_{i} \right | + \left ( 1-\lambda  \right )\sum_{i=1}^{j}\: a_{i}^{2} \, \leq \tau
$$

For our purposes I will vary alpha between 0.01 and 1.0. The same scaled dataset from the Lasso regression section will be used.

***Disclaimer:*** depending on your computer, this may take a second or two to run.

In [56]:
r_squared_tracker <- rep(0,100)

for (i in 1:100) {
    
    elastic_net_regression <- cv.glmnet(x=feature_matrix, y=response_matrix, alpha=i/100, nfolds=5, type.measure='mse',family='gaussian')
    r_squared_tracker[i] <- elastic_net_regression$glmnet.fit$dev.ratio[which(elastic_net_regression$glmnet.fit$lambda == elastic_net_regression$lambda.min)]
}




### Best value of alpha

Let's find the maximum R^2 value of our r_squared_tracker and report the corresponding alpha value.



In [59]:
max(r_squared_tracker)

[1] 0.8029778

In [61]:
optimal_alpha <- (which.max(r_squared_tracker)-1)/length(r_squared_tracker)
optimal_alpha

[1] 0.4



This is by far the best r_squared value we have been able to gleam from this particular dataset. However our results are more then likely overfitting to random effects. Let's rerun the cross-validated elastic net using our optimal alpha value.





### Cross-validated elastic net using optimal alpha value



In [65]:
elastic_net_regression_optimal_alpha <- cv.glmnet(x=feature_matrix, y=response_matrix, alpha=optimal_alpha, nfolds=5, type.measure='mse',family='gaussian')



### Optimal alpha value elastic net model coefficients



In [66]:
coef(elastic_net_regression_optimal_alpha, s= elastic_net_regression_optimal_alpha$lambda.min)

16 x 1 sparse Matrix of class "dgCMatrix"
                     1
(Intercept) 886.836984
M            89.861035
So           53.603859
Ed          135.649191
Po1         218.664243
Po2          65.166278
LF            4.804134
M.F          62.972357
Pop           .       
NW           17.172927
U1          -51.133968
U2           87.786691
Wealth       27.001069
Ineq        188.509244
Prob        -86.992561
Time          .       



Notice the optimal elastic net model is using more of the predictive features then either the Lasso regression or the stepwise regression. To do a proper comparison between the three models let's run our standard LOOCV regression model using these predictive features.



In [69]:
reduced_regression_elastic_net <- train(Crime ~ M+So+Ed+Po1+Po2+LF+M.F+NW+U1+U2+Wealth+Ineq+Prob,data,method = "lm",trControl=train_control)
reduced_regression_elastic_net

Linear Regression 

47 samples
13 predictors

No pre-processing
Resampling: Leave-One-Out Cross-Validation 
Summary of sample sizes: 46, 46, 46, 46, 46, 46, ... 
Resampling results:

  RMSE      Rsquared   MAE     
  254.0771  0.5830473  199.5026

Tuning parameter 'intercept' was held constant at a value of TRUE

In [ ]:
summary(reduced_regression_elastic_net)



Even more dismal then the optimal model chosen by Lasso regression. However, if we remove the non-significant features from our model we notice the same set of predictive features gleamed from our stepwise-regression model:

* Po1
* Ineq
* Ed
* M
* Prob
* U2



### Fin!

In this notebook three different approaches to predictive feature selection were demonstrated:
1. Stepwise regression
2. Lasso regression
3. Elastic Net

Each of the three approaches has their strengthes and weaknesses, but as a general rule of thumb, stepwise regression and other greedy predictive feature selection approaches like backwards or forward elimination should be used for exploratory analysis or on small datasets like this one that are not too computationally expensive. Slower models like Lasso or elastic net generally provide better results, but at a higher computational cost. Leveraging all three models allows for the optimal selection of predictive features.